In [1]:
import torch as tc
import numpy as np
import pickle
import os
from wish_utils import *
from WishDataSet import WishTubeFrameDataSet, get_transform, WishWorkspaceDataSet
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import utils
import threading
import multiprocessing as mp
from tqdm.notebook import tqdm

In [2]:
model_path = '/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/tube_by_tube_withneighbors_fasterrcnn_resnet50_fpn.pt'

In [3]:
device = tc.device('cuda') if tc.cuda.is_available() else tc.device('cpu')
device

device(type='cpu')

In [4]:
workspace_path = '/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00042730.npz'

In [5]:
ds = WishWorkspaceDataSet(workspace_path)

data set create for /mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00042730.npz shape(1520, 128, 1540)


In [6]:
len(ds)

1520

In [7]:
x = ds[0]
x.shape, type(x), x.device

(torch.Size([3, 128, 1540]), torch.Tensor, device(type='cpu'))

In [8]:
data_loader = tc.utils.data.DataLoader(ds, batch_size=8, shuffle=False, num_workers=4)

In [9]:
len(data_loader)

190

In [10]:
tc.cuda.is_available() , tc.cuda.device_count()

(False, 0)

In [11]:
model = get_object_detection_model(num_classes=2)
model.load_state_dict(tc.load(model_path, map_location=device))
model = model.to(device)

/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# ws = np.load(workspace_path)['arr_0']

In [13]:
# ws.shape

In [12]:
def plot_tube_detections(tube_idx, bin_data, predicted_boxes, score_threshold = 0.75):
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    cs = plt.contourf(bin_data)
    plt.colorbar(cs)
    
    plotted_predictions = 0
    for box, score in zip(predicted_boxes['boxes'], predicted_boxes['scores']):
        # print(f"prediction={box} score={score}")
        x_hat, y_hat = (box[0]+box[2])/2, (box[1]+box[3])/2
        if score < score_threshold:
            # print(f"----->Ignoring the prediction at x={x_hat} y={y_hat} with score={score}")
            plt.plot(x_hat, y_hat, marker="x", markersize=5, markeredgecolor="white", markerfacecolor="white")
            continue
            
        # print(f"Prediction coordinate x={x_hat} y={y_hat}")
        plt.plot(x_hat, y_hat, marker="x", markersize=5, markeredgecolor="red", markerfacecolor="red")
        plotted_predictions += 1

    plt.title(f"Tube-{tube_idx} Predicted-{plotted_predictions} peaks")
    plt.show()

In [15]:
# frame_to_tensor = A.Compose([ToTensorV2(p=1.0)])

In [16]:
# img = np.tile(frame_data[:, :, None], 3).astype(np.float32)

In [17]:
# b = a(image=img)

In [18]:
# b['image'].shape, type(b['image'])

In [19]:
# for tube in range(ws.shape[0]):
#     frame_data = ws[tube, ...]
#     img = np.tile(frame_data[:,:, None], 3).astype(np.float32)
#     img = frame_to_tensor(image=img)
#     # print(np.tile(frame_data[None,:,:], 3).shape)
#     # img_res = tc.from_numpy(img_res)

In [20]:
# img

In [21]:
# model.eval()

# probe = 0

# for img in range(ws.shape[0]):
#     frame_data = ws[tube, ...]
#     frame = np.tile(frame_data[:,:, None], 3).astype(np.float32)
#     frame = frame_to_tensor(image=frame)
#     with tc.no_grad():
#         prediction = model([frame['image'].to(device)])[0]
#         print(prediction)
#         nms_prediction = apply_nms(prediction, iou_thresh=0.001)
#         plot_tube_detections(frame['image'][0,:,:], nms_prediction, 0.50)
#         probe += 1
#         if probe > 1:
#             break

In [13]:
%%time
model.eval()
probe = 0
predicted_indices_with_score = []

for batch_idx, img_batch in enumerate(data_loader):
    for img_idx, img in enumerate(img_batch):
        tube_idx = batch_idx * data_loader.batch_size + img_idx
        with tc.no_grad():
            prediction = model([img.to(device)])[0]
            nms_prediction = apply_nms(prediction, iou_thresh=0.001)
            for box, score in zip(nms_prediction['boxes'], nms_prediction['scores']):
                tof = (box[0]+box[2])/2
                tube_res = (box[1]+box[3])/2
                predicted_indices_with_score.append([tube_idx, tube_res, tof, score])
            # plot_tube_detections(tube_idx, img[0,:,:], nms_prediction, 0.50)
            probe += 1
            
    # if probe > 1:
    #     break

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


CPU times: user 1h 24min 35s, sys: 2min 36s, total: 1h 27min 11s
Wall time: 14min 37s


In [14]:
len(predicted_indices_with_score)

391

In [15]:
pred_cpu = []
for pred in predicted_indices_with_score:
    pred_cpu.append([pred[0], pred[1].cpu(), pred[2].cpu(), pred[3].cpu()])

pred_cpu

[[12, tensor(56.6820), tensor(654.7282), tensor(0.8526)],
 [13, tensor(55.3395), tensor(667.3365), tensor(0.2825)],
 [40, tensor(22.5053), tensor(297.8273), tensor(0.9536)],
 [98, tensor(66.9794), tensor(433.3065), tensor(0.8417)],
 [99, tensor(67.0536), tensor(440.5806), tensor(0.1845)],
 [127, tensor(79.0471), tensor(691.3815), tensor(0.9670)],
 [128, tensor(79.6929), tensor(696.4612), tensor(0.9995)],
 [128, tensor(79.6868), tensor(327.7531), tensor(0.8914)],
 [129, tensor(78.7200), tensor(700.8126), tensor(0.9992)],
 [129, tensor(20.1835), tensor(545.9242), tensor(0.1282)],
 [130, tensor(21.7909), tensor(549.5570), tensor(0.9967)],
 [131, tensor(21.2508), tensor(553.3251), tensor(0.9985)],
 [132, tensor(23.2454), tensor(556.7203), tensor(0.9783)],
 [151, tensor(91.3288), tensor(494.7783), tensor(0.3946)],
 [152, tensor(85.9346), tensor(499.5344), tensor(0.7858)],
 [152, tensor(125.3116), tensor(1488.7744), tensor(0.1874)],
 [153, tensor(125.8554), tensor(1488.8042), tensor(0.0893)]

In [19]:
pred_cpu=np.array(pred_cpu)

In [25]:
len(pred_cpu[pred_cpu[:, -1] < 0.5])

98

In [28]:
np.savez_compressed('/mnt/ceph/home/wj1132075/Desktop/CNN_Model_Data/Inferences/RealData/WISH00042730_inferences_trained_with_neighbors.npz', predicted_indices_with_score)

In [17]:
arr = np.array(predicted_indices_with_score)
arr.shape

(421, 4)

In [18]:
len(arr[arr[:, -1]>=0.5])

287

### Data parallelism to speedup

In [13]:
def doInference(img_batch):
    model.eval()
    predicted_indices_with_score = []
    batch_idx, img_batch = img_batch
    print(f"Starting {batch_idx}")
    for img_idx, img in enumerate(img_batch):
        tube_idx = batch_idx * batch_size + img_idx
        with tc.no_grad():
            prediction = model([img.to(device)])[0]
            nms_prediction = apply_nms(prediction, iou_thresh=0.001)
            for box, score in zip(nms_prediction['boxes'], nms_prediction['scores']):
                tof = (box[0]+box[2])/2
                tube_res = (box[1]+box[3])/2
                predicted_indices_with_score.append([tube_idx, tube_res, tof, score])
                # print(f"Inferred batch={batch_idx} tube={tube_idx}")
                
    print(f"Returning results for batch={batch_idx}...........")
    return predicted_indices_with_score

In [14]:
%%time
data_loader = tc.utils.data.DataLoader(ds, batch_size=8, shuffle=False, num_workers=2)
batch_size = data_loader.batch_size
pool = mp.Pool(processes=2)

inferences = pool.map(doInference, enumerate(data_loader))

pool.close()
print("pool closed, waiting to join...")
pool.join()
print(len(inferences))

Starting 0Starting 24

Starting 48Starting 72



[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Returning results for batch=72...........
Starting 73
Returning results for batch=48...........
Starting 49
Returning results for batch=73...........
Starting 74
Returning results for batch=49...........
Starting 50
Returning results for batch=50...........
Starting 51
Returning results for batch=74...........
Starting 75
Returning results for batch=75...........
Starting 76
Returning results for batch=51...........
Starting 52
Returning results for batch=76...........
Starting 77
Returning results for batch=52...........
Starting 53
Returning results for batch=77...........
Starting 78
Returning results for batch=53...........
Starting 54
Returning results for batch=78...........
Starting 79
Returning results for batch=54...........
Starting 55
Returning results for batch=79...........
Starting 80
Returning results for batch=55...........
Starting 56
Returning results for batch=80...........
Starting 81
Returning results for batch=56...........
Starting 57
Returning results for batch=

Process ForkPoolWorker-34:
Process ForkPoolWorker-33:
Traceback (most recent call last):
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)


KeyboardInterrupt: 

  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/wj1132075/.conda/envs/cuda_1/lib/python3.10/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
KeyboardInterrupt
  File "/home/wj1132075/.conda/envs/cuda_1/lib/pyt

In [20]:
things=[7,5,1,6,4,2]

def worker(thing):
    id, n = thing
    # time.sleep(n)
    print(f"id={id} thing={n} pid={os.getpid()}\n")
    return n
    
pool = mp.Pool(processes=mp.cpu_count())
results = pool.map(worker, enumerate(things))
pool.close()
pool.join()
for r in results:
    print(f"Result {r}") 

id=2 thing=1 pid=313274

id=5 thing=2 pid=313278

id=4 thing=4 pid=313277

id=1 thing=5 pid=313273

id=3 thing=6 pid=313276

id=0 thing=7 pid=313272

Result 7
Result 5
Result 1
Result 6
Result 4
Result 2
